In [1]:
import numpy as np
import sys
from collections import Counter
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
import math
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import contractions
import re
import spacy
from spacy import displacy
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from matplotlib import rcParams
import matplotlib.pyplot as plt
import seaborn as sns
import time
import keras
from keras.models import Model
from keras.layers import Input
from keras import layers
from keras.layers import Dense

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rohiljaveri/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rohiljaveri/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/rohiljaveri/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
STOP_WORDS = set(stopwords.words('english'))
NER = spacy.load("en_core_web_sm")

In [3]:
# Reading in dataset
yelp_df = pd.read_csv("yelp.csv")
yelp_df = yelp_df[['review_id', 'text', 'stars']]
yelp_df['stars'] = yelp_df['stars'].apply(str)
yelp_df.head()

,review_id,text,stars
0,fWKvX83p0-ka4JS3dc6E5A,My wife took me here on my birthday for breakf...,5
1,IjZ33sJrzXqU-0X6U8NwyA,I have no idea why some people give bad review...,5
2,IESLBzqUCLdSzSqm0eCSxQ,love the gyro plate. Rice is so good and I als...,4
3,G-WvGaISbqqaMHlNnByodA,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",5
4,1uJFq2r5QfJG_6ExMRCaGw,General Manager Scott Petello is a good egg!!!...,5


In [4]:
# Reading in dataset
movie_df = pd.read_csv("movie.csv")
movie_df.head()
movie_df = movie_df[['text', 'label']]
# yelp_df['stars'] = yelp_df['stars'].apply(str)
movie_df.head()


,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [5]:
def apply_space_tokenization(text):
    new_text = text.split(' ')
    return new_text

def apply_lowercase(text):
    new_text = text.lower()
    return new_text

In [6]:
yelp_df['text'] = yelp_df['text'].apply(apply_lowercase)
#yelp_df['text'] = yelp_df['text'].apply(apply_space_tokenization)
yelp_df

,review_id,text,stars
0,fWKvX83p0-ka4JS3dc6E5A,my wife took me here on my birthday for breakf...,5
1,IjZ33sJrzXqU-0X6U8NwyA,i have no idea why some people give bad review...,5
2,IESLBzqUCLdSzSqm0eCSxQ,love the gyro plate. rice is so good and i als...,4
3,G-WvGaISbqqaMHlNnByodA,"rosie, dakota, and i love chaparral dog park!!...",5
4,1uJFq2r5QfJG_6ExMRCaGw,general manager scott petello is a good egg!!!...,5
...,...,...,...
9995,Ubyfp2RSDYW0g7Mbr8N3iA,first visit...had lunch here today - used my g...,3
9996,2XyIOQKbVFb6uXQdJ0RzlQ,should be called house of deliciousness!\n\ni ...,4
9997,jyznYkIbpqVmlsZxSDSypA,i recently visited olive and ivy for business ...,4
9998,5UKq9WQE1qQbJ0DJbc-B6Q,my nephew just moved to scottsdale recently so...,2


In [7]:
movie_df['text'] = movie_df['text'].apply(apply_lowercase)
#yelp_df['text'] = yelp_df['text'].apply(apply_space_tokenization)
movie_df

,text,label
0,i grew up (b. 1965) watching and loving the th...,0
1,"when i put this movie in my dvd player, and sa...",0
2,why do people who do not know what a particula...,0
3,even though i have great interest in biblical ...,0
4,im a die hard dads army fan and nothing will e...,1
...,...,...
39995,"""western union"" is something of a forgotten cl...",1
39996,this movie is an incredible piece of work. it ...,1
39997,my wife and i watched this movie because we pl...,0
39998,"when i first watched flatliners, i was amazed....",1


# Preprocessing Expirements
#### Potentially we do some constants throughout our experiments? I.e Tokenize just by spaces, and lowecase all

### No Pre-Processing

### Lemanization

In [8]:
def apply_lem(text):
    lem = WordNetLemmatizer()
    new_text = []
    for word in text.split(' '):
        new_word = lem.lemmatize(word)
        new_text.append(new_word)
    review = ' '.join(new_text)
    return review

### Stemming

In [9]:
def apply_stemming(text):
    ps = PorterStemmer()
    new_text = []
    for word in text.split(' '):
        new_word = ps.stem(word)
        new_text.append(new_word)
    review = ' '.join(new_text)
    return review

### Contraction expansion

In [10]:
def apply_contraction_expansion(text):
    new_text = []
    for word in text.split(' '):
        new_word = contractions.fix(text)
        new_text.append(new_word)
    review = ' '.join(new_text)
    return review

### Removing Number related text

In [11]:
def apply_number_removal(text):
    review = re.sub(r'[0-9]', '', text)
    return review

### Punctuation Removal

In [12]:
def apply_punctuation_removal(text):
    review = re.sub(r'[.,!?-_;:&#$@%=+]', '', text)
    return review

### Stopword Removal

In [13]:
def apply_stopword_removal(text):
    new_text = []
    for word in text.split(' '):
        if text in STOP_WORDS:
            continue
        else:
            new_text.append(word)
    review = ' '.join(new_text)
    return review

### Spell Check Removal

In [14]:
def apply_spelling_correction(text):
    textBlb = TextBlob(text)
    textCorrected = textBlb.correct() 
    return textCorrected

### NER AND POS TAGGING

In [15]:
def apply_spacy(text):
    applied_text = NER(text)
    return applied_text


def get_pos(spacy_text):
    print(spacy_text)
    new_sentence = []
    excluded_tags = {"NOUN", "PRON", "PROPN", "DET", "ADP", "PUNCT",}
    for word in spacy_text:
        if word.pos_ not in excluded_tags:
            new_sentence.append(word.text)
    review = ' '.join(new_sentence)
    return review

# def get_ner(spacy_text):
#     for word in spacy_text:
        
#         print(word.text, word.label_)
        
# to return the labels
# spacy_words = apply_spacy("I honestly hated the play, it was so boring")   
# get_pos(spacy_words)

# for word in spacy_words.ents:
#     print(word.text,word.label_)




### Removing Certain P.O.S

### Pre-Processing Combination
#### Perhaps we run experiments with random combinations of all methods and see best performance

# Model Creation & Test Execution

In [20]:
def report_metrics(gold_labels, predicted_labels, name, clf):
    # Reporting metrics & confusion matrix
    acc = round(accuracy_score(gold_labels, predicted_labels), 4)
    err = round(1 - acc, 4)
    pre = round(precision_score(gold_labels, predicted_labels, average='macro'), 4)
    rec = round(recall_score(gold_labels, predicted_labels, average='macro'), 4)
    f1 = round(f1_score(gold_labels, predicted_labels, average='macro'), 4)

    print("Accuracy: ", acc)
    print("Error: ", err)
    print("Precision: ", pre)
    print("Recall: ", rec)
    print("F1-Score: ", f1)

    cf_matrix = confusion_matrix(gold_labels, predicted_labels, labels=clf.classes_)
    rcParams['figure.figsize'] = 16,12
    sns.heatmap(cf_matrix, annot=True, cmap="Greens", xticklabels=clf.classes_, yticklabels=clf.classes_, fmt='g')
    plt.title(name)
    plt.ylabel("True Labels")
    plt.xlabel("Predicted Labels")
    plt.show()
    
    return acc, err, pre, rec, f1

In [21]:
def setup_experiment(data, pre_processing_step):
    print(pre_processing_step)
    experiment_df = data
    start_time = time.time()
    experiment_df['text'] = experiment_df['text'].apply(pre_processing_step)
    end_time = time.time()
    
    # Creating a corpus (list of documents)
    corpus = []
    for idx, row in experiment_df.iterrows():
        corpus.append(row['text'])
    
    # Using tf-idf representation as a baseline featutre representation
    vectorizer = CountVectorizer()
    #vectorizer = TfidfVectorizer(max_features=400)
    
    X = vectorizer.fit_transform(corpus).toarray()
    y = experiment_df['label']
    
    return X, y, start_time - end_time

In [24]:
def run_experiment(X, y, model_name, time):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=18)
    
    if model_name == 'NB':
        model = MultinomialNB(fit_prior=False)
    elif model_name == 'Deep':
        model = keras.Sequential([
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),])
        model.compile(loss="mean_squared_error", optimizer="adam")   
    else:
        pass
    
    # Running experiment
    model.fit(X_train, y_train)
    
    # Make predictions
    predictions = model.predict(X_test)
    
    test_acc = model.evaluate(X_test, y_test, verbose=0)
    test_error = 1 - test_acc
    print(test_acc, test_error)
#     acc, err, pre, rec, f1 = report_metrics(y_test, predictions, model_name, model)
    
    return test_acc, test_error

In [25]:
# Pre-processing options
# apply_spelling_correction
options = [apply_lem, apply_stemming, apply_contraction_expansion, apply_number_removal,
          apply_punctuation_removal, apply_stopword_removal]
options_string = ['lem', 'stemming', 'contraction_expansion', 'number_removal',
          'punctuation_removal', 'stopword_removal']

columns = ['Pre-Processing', 'Acuracy', 'Error', 'Precision', 'Recall', 'F1-Score', 'Time']

results_df = pd.DataFrame(columns=columns)

for option, name in zip(options, options_string):
    X, y, t = setup_experiment(movie_df, option)
    acc, err = run_experiment(X, y, "Deep", t)
#     acc, err, pre, rec, f1, t = run_experiment(X, y, "Deep", t)
    df_dict = {'Pre-Processing':name, 'Acuracy':acc, 'Error':err, 'Precision':pre, 'Recall':rec, 'F1-Score':f1, 'Time':t}
    results_df = results_df.append(df_dict, ignore_index=True)
    
for option, name in zip(options, options_string):
    X, y, t = setup_experiment(movie_df, option)
    acc, err, pre, rec, f1, t = run_experiment(X, y, "NB", t)
    df_dict = {'Pre-Processing':name, 'Acuracy':acc, 'Error':err, 'Precision':pre, 'Recall':rec, 'F1-Score':f1, 'Time':t}
    results_df = results_df.append(df_dict, ignore_index=True)

<function apply_lem at 0x7fe393062320>
1000/1000 [==============================] - 17s 17ms/step - loss: 0.2865
0.2499626725912094 0.7500373274087906


NameError: name 'pre' is not defined

In [ ]:
results_df

In [19]:
execute_all = False
if execute_all:
    experiment_df = yelp_df
    experiment_df['text'] = experiment_df['text'].apply(apply_lem)
    print(1)
    experiment_df['text'] = experiment_df['text'].apply(apply_stemming)
    print(2)
    experiment_df['text'] = experiment_df['text'].apply(apply_contraction_expansion)
    print(3)
    experiment_df['text'] = experiment_df['text'].apply(apply_number_removal)
    print(4)
    experiment_df['text'] = experiment_df['text'].apply(apply_punctuation_removal)
    print(5)
    experiment_df['text'] = experiment_df['text'].apply(apply_stopword_removal)
    print(6)

    # Creating a corpus (list of documents)
    corpus = []
    for idx, row in experiment_df.iterrows():
        corpus.append(row['text'])

    # Using tf-idf representation as a baseline featutre representation
    vectorizer = CountVectorizer()
    #vectorizer = TfidfVectorizer(max_features=400)

    X = vectorizer.fit_transform(corpus).toarray()
    y = experiment_df['stars']

    run_experiment(X, y, "NB")

In [20]:
# Two Models

# Logistic Regression


# Deep Learning (idk steal one of the previous ones we made)

# Result Collection & Visualization